In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !pip install torchmetrics
# !git clone https://github.com/Hpeox/Intellectual_manifest.git

In [4]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
# from tqdm import tqdm

import cv2, time
# from PIL import Image
# from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchmetrics
import torchvision.models as models
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

import albumentations as A


# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti Laptop GPU'

In [6]:
# 读取数据集
train_path = glob.glob("./data/sample/*/*")
test_path = glob.glob("./data/test/*")

train_path.sort()
test_path.sort()

# train_df = pd.read_csv("data/train.csv")
# train_df = train_df.sort_values(by="name")
# train_label = train_df["label"].values

train_label = [np.int64(0) for i in range(480)]+[np.int64(1) for i in range(2400)]

# 自定义数据集
# 带有图片缓存的逻辑
DATA_CACHE = {}


class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            img = cv2.imread(self.img_path[index])
            DATA_CACHE[self.img_path[index]] = img
        if self.transform is not None:
            img = self.transform(image=img)["image"]
        img = img.transpose([2, 0, 1])
        return img, torch.from_numpy(np.array(self.img_label[index]))

    def __len__(self):
        return len(self.img_path)

dataset = XunFeiDataset(
    train_path,
    train_label,
    A.Compose(
        [
            A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    ),
)
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset=dataset, lengths=[0.9, 0.1], generator=torch.Generator().manual_seed(42)
)

# 训练集
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=False
)

# 验证集
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=False
)

# 测试集
test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(
        test_path,
        [0] * len(test_path),
        A.Compose(
            [
                A.Resize(256, 256),
                A.RandomCrop(224, 224),
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.5),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ]
        ),
    ),
    batch_size=50,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)

In [7]:
class XunFeiNet50(nn.Module):
    def __init__(self):
        super(XunFeiNet50, self).__init__()
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet101(nn.Module):
    def __init__(self):
        super(XunFeiNet101, self).__init__()
        model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet152(nn.Module):
    def __init__(self):
        super(XunFeiNet152, self).__init__()
        model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet34(nn.Module):
    def __init__(self):
        super(XunFeiNet34, self).__init__()
        model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet18(nn.Module):
    def __init__(self):
        super(XunFeiNet18, self).__init__()
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet121(nn.Module):
    def __init__(self):
        super(XunFeiNet121, self).__init__()
        model = models.densenet121(num_classes=2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

model121 = XunFeiNet121().to(device)
model18 = XunFeiNet18().to(device)
model34 = XunFeiNet34().to(device)
model50 = XunFeiNet50().to(device)
model101 = XunFeiNet101().to(device)
model152 = XunFeiNet152().to(device)
# model = model.to(device)
criterion = nn.CrossEntropyLoss().cuda()

In [8]:
# 模型训练
def train(train_loader, model, criterion, optimizer):
    start = time.time()
    # start_batch = [start, 0]
    model.train()
    train_loss = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    for _, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if (i + 1) % 30 == 0:
        #     start_batch[(1+(i + 1) // 100) % 2] = time.time()
        #     print(
        #         "Train loss",
        #         loss.item(),
        #         "t={}s".format(
        #             start_batch[(1+(i + 1) // 100) % 2]
        #             - start_batch[((i + 1) // 100) % 2]
        #         ),
        #     )

        preds = torch.cat((preds, output.cpu().argmax(1)))
        target_all = torch.cat((target_all, target.cpu()))

        train_loss += loss.item()

    val_acc = torchmetrics.functional.classification.multiclass_f1_score(
        preds, target_all, num_classes=2, average="macro"
    )
    print("t={}s".format(time.time() - start))
    print("F1 score", val_acc)
    return train_loss / len(train_loader)


# 模型验证
def validate(val_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    val_loss=0.0
    with torch.no_grad():
        for _, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = criterion(output, target)
            # val_acc += (output.argmax(1) == target).sum().item()
            preds = torch.cat((preds, output.cpu().argmax(1)))
            target_all = torch.cat((target_all, target.cpu()))

        val_acc = torchmetrics.functional.classification.multiclass_f1_score(
            preds, target_all, num_classes=2, average="macro"
        )
        val_loss += loss.item()
    # return val_acc / len(val_loader.dataset)
    return val_acc,val_loss/len(val_loader)


# 模型预测
def predict(test_loader, model):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())

    return np.vstack(test_pred)

In [ ]:
epochs = 50
print("========resnet18========")
model=model18
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss, "\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_18.pth")
print("========resnet34========")
model=model34
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_34.pth")

print("========resnet50========")
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_50.pth")

print("========resnet101========")
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_101.pth")

print("========resnet152========")
model=model152
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_152.pth")

print("========densnet121========")
model=model121
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_121.pth")

========resnet18========
t=24.906310319900513s
F1 score tensor(0.6211)
epoch 1 : train_loss: 0.541062899479052 val_loss: 0.08855052590370179 
val_f1: tensor(0.7541) 

t=10.155979633331299s
F1 score tensor(0.7262)
epoch 2 : train_loss: 0.4600911402120823 val_loss: 0.08177627921104431 
val_f1: tensor(0.7360) 

t=10.204305410385132s
F1 score tensor(0.7476)
epoch 3 : train_loss: 0.40938178286319826 val_loss: 0.06173466444015503 
val_f1: tensor(0.8046) 

t=10.660547018051147s
F1 score tensor(0.7851)
epoch 4 : train_loss: 0.36282722324859806 val_loss: 0.05929139256477356 
val_f1: tensor(0.8402) 

t=11.069799184799194s
F1 score tensor(0.7950)
epoch 5 : train_loss: 0.3364242286216922 val_loss: 0.04950132369995117 
val_f1: tensor(0.8189) 

t=11.352064847946167s
F1 score tensor(0.8193)
epoch 6 : train_loss: 0.2989829767767976 val_loss: 0.05007819533348083 
val_f1: tensor(0.8815) 

t=10.538376331329346s
F1 score tensor(0.8249)
epoch 7 : train_loss: 0.28079985663658236 val_loss: 0.0510823726654052

In [9]:
epochs = 50
# print("========resnet18========")
# model=model18
# model.load_state_dict(torch.load("./model/baseline25_18.pth"))
# optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
# for epoch in range(epochs):
#     train_loss = train(train_loader, model, criterion, optimizer)
#     val_acc,val_loss = validate(val_loader, model, criterion)
#     print(
#         "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss, "\nval_f1:", val_acc, "\n"
#     )
# torch.save(model.state_dict(),"./model/baseline25_18.pth")
# print("========resnet34========")
# model=model34
# optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
# model.load_state_dict(torch.load("./model/baseline25_34.pth"))
# for epoch in range(epochs):
#     train_loss = train(train_loader, model, criterion, optimizer)
#     val_acc,val_loss = validate(val_loader, model, criterion)
#     print(
#         "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
#     )
# torch.save(model.state_dict(),"./model/baseline25_34.pth")

print("========resnet50========")
f1_50=[]
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_50.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_50.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_50.pth")

print("========resnet101========")
f1_101=[]
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_101.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_101.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_101.pth")

print("========resnet152========")
model=model152
f1_152=[]
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_152.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_152.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_152.pth")

print("========densnet121========")
f1_121=[]
model=model121
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_121.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_121.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_121.pth")

========resnet50========
t=18.050381898880005s
F1 score tensor(0.9495)
epoch 1 : train_loss: 0.0691385610535005 val_loss: 0.00710305737124549 
val_f1: tensor(0.9329) 

t=119.51899600028992s
F1 score tensor(0.9540)
epoch 2 : train_loss: 0.0672593042369426 val_loss: 0.005375111268626319 
val_f1: tensor(0.9307) 

t=13.315320253372192s
F1 score tensor(0.9534)
epoch 3 : train_loss: 0.06983617539115158 val_loss: 0.012161867486106025 
val_f1: tensor(0.9442) 

t=84.046071767807s
F1 score tensor(0.9560)
epoch 4 : train_loss: 0.06861240846783291 val_loss: 0.0017942395061254501 
val_f1: tensor(0.9433) 

t=13.68542218208313s
F1 score tensor(0.9623)
epoch 5 : train_loss: 0.057293530760539904 val_loss: 0.0019546347773737377 
val_f1: tensor(0.9433) 

t=13.439164638519287s
F1 score tensor(0.9540)
epoch 6 : train_loss: 0.06320193257573763 val_loss: 0.0031554334693484837 
val_f1: tensor(0.9442) 

t=13.498860597610474s
F1 score tensor(0.9619)
epoch 7 : train_loss: 0.06164775211593987 val_loss: 0.01051984

In [14]:
f1_152

[tensor(0.9125),
 tensor(0.9285),
 tensor(0.9262),
 tensor(0.9220),
 tensor(0.9125),
 tensor(0.9385),
 tensor(0.9085),
 tensor(0.9285),
 tensor(0.9085),
 tensor(0.9262),
 tensor(0.9433),
 tensor(0.9508),
 tensor(0.9250),
 tensor(0.9364),
 tensor(0.8983),
 tensor(0.9375),
 tensor(0.9523),
 tensor(0.9237),
 tensor(0.9250),
 tensor(0.9516),
 tensor(0.9442),
 tensor(0.9508),
 tensor(0.9307),
 tensor(0.9451),
 tensor(0.9250),
 tensor(0.9375),
 tensor(0.9307),
 tensor(0.9566),
 tensor(0.9340),
 tensor(0.9385),
 tensor(0.9508),
 tensor(0.9375),
 tensor(0.9580),
 tensor(0.9573),
 tensor(0.9643),
 tensor(0.9274),
 tensor(0.9508),
 tensor(0.9580),
 tensor(0.9237),
 tensor(0.9329),
 tensor(0.9442),
 tensor(0.9442),
 tensor(0.9559),
 tensor(0.9395),
 tensor(0.9508),
 tensor(0.9500),
 tensor(0.9637),
 tensor(0.9631),
 tensor(0.9573),
 tensor(0.9643)]

101/152/121 100epochs f1_marco~0.96

In [ ]:
# for best model
model=model152
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_152.pth")

In [ ]:
# for best model
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_101.pth")

In [ ]:
# for best model
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_50.pth")

Train loss 0.5575281381607056 t=4.891497373580933s
Train loss 0.5705251097679138 t=10.128498792648315s


In [ ]:
# for best model
model=model34
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_34.pth")

In [ ]:
# 对测试集多次预测
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model152
model.load_state_dict(torch.load("./model/baseline_152.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.9*predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model34
model.load_state_dict(torch.load("./model/baseline_34.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.9*predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit6.csv', index=None)

In [ ]:
model.train()
train_loss = 0.0
model = model.to(device)
for i, (input, target) in enumerate(train_loader):
    input = input.to(device)
    target = target.to(device)
    output = model(input)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
epochs=4
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_test.pth")



In [ ]:
epochs=3
model101= XunFeiNet().to(device)
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_101.pth")

In [ ]:
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_test.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit5.csv', index=None)

In [ ]:
model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
model.fc

In [ ]:
val_acc = validate(val_loader, model)
val_acc

In [ ]:
dataset=XunFeiDataset(train_path[:-1000], train_label[:-1000],
            A.Compose([
            # A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            # A.HorizontalFlip(p=0.5),
            # A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        )
train_dataset, test_dataset=torch.utils.data.random_split(dataset=dataset,lengths=[0.9,0.1],generator=torch.Generator().manual_seed(42))
train_dataset

In [ ]:
model.load_state_dict(torch.load("./model/baseline_test.pth"))
val_acc,val_loss = validate(val_loader, model, criterion)
print(val_loss)
len(val_loader.dataset)